In [1]:
import time
from tqdm import tqdm

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms.autoaugment import AutoAugmentPolicy
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import timm
import model.swin_v1 as swin
from timm.data import Mixup

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
unique_idx = '4'

batch_size = 512
num_workers = 8
img_size = 224
patch_size = 16
dropout = 0.1

model_name = 'swin_tiny_patch4_window7_224.ms_in22k'
num_classes = 10

device = f'cuda:{unique_idx}'

label_smoothing = 0.0
learning_rate = 1e-5
epochs = 30

model_path = f'../models/swin/finetune_model:{unique_idx}.pth'  # 모델 저장 경로

In [3]:
# 데이터 증강을 위한 전처리
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(img_size),  # 무작위 크기 및 비율로 자르기
    transforms.AutoAugment(AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
model1 = swin.SwinTransformer(img_size=224, num_classes=num_classes, drop_path_rate=0.1).to(device)
model2 = swin.SwinTransformer(img_size=224, drop_path_rate=0.1)
model2.load_state_dict(torch.load('../models/swin/model.pth', map_location=device))
model2.classifier = nn.Linear(model2.classifier.in_features, num_classes)
model2.to(device)
model3 = timm.create_model(model_name=model_name, 
                          pretrained=False, 
                          num_classes=num_classes).to(device)
model4 = timm.create_model(model_name=model_name, 
                          pretrained=False, 
                          num_classes=100)

model4.load_state_dict(torch.load('../models/swin/test_model.pth', map_location=device))
model4.head.fc = nn.Linear(model4.head.fc.in_features, num_classes)
model4.to(device)

model5 = timm.create_model(model_name=model_name, 
                          pretrained=True, 
                          num_classes=num_classes).to(device)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

optimizer1 = optim.AdamW(model1.parameters(), lr=learning_rate, weight_decay=1e-8)
optimizer2 = optim.AdamW(model2.parameters(), lr=learning_rate, weight_decay=1e-8)
optimizer3 = optim.AdamW(model3.parameters(), lr=learning_rate, weight_decay=1e-8)
optimizer4 = optim.AdamW(model4.parameters(), lr=learning_rate, weight_decay=1e-8)
optimizer5 = optim.AdamW(model5.parameters(), lr=learning_rate, weight_decay=1e-8)

In [ ]:
mixup_fn = Mixup(mixup_alpha=1.0, 
                 cutmix_alpha=1.0, 
                 prob=1.0, 
                 switch_prob=0.5, 
                 mode='batch',
                 label_smoothing=0.1,
                 num_classes=100)

In [6]:
training_time = 0
losses = []
val_losses = []

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(epochs):
    model1.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for i, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer1.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model1(inputs)
            loss = criterion(outputs, labels)

        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer1)
        clip_grad_norm_(model1.parameters(), max_norm=1.0)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer1)
        scaler.update()
        # scheduler.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

 
    model1.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model1(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, Duration: {epoch_duration:.2f} sec'
    print(text)

Epoch 1: 100%|██████████| 98/98 [00:59<00:00,  1.66it/s]


	Loss: 2.17409602476626, Val Loss: 1.9592431485652924, Duration: 70.38 sec


Epoch 2: 100%|██████████| 98/98 [00:59<00:00,  1.66it/s]


	Loss: 2.1062114968591805, Val Loss: 1.8685737609863282, Duration: 70.60 sec


Epoch 3: 100%|██████████| 98/98 [00:58<00:00,  1.66it/s]


...

	Loss: 1.6509946827985802, Val Loss: 1.254001396894455, Duration: 70.65 sec


Epoch 29: 100%|██████████| 98/98 [00:59<00:00,  1.65it/s]


	Loss: 1.6367833285915607, Val Loss: 1.2427599906921387, Duration: 71.10 sec


Epoch 30: 100%|██████████| 98/98 [00:59<00:00,  1.65it/s]


	Loss: 1.632750312892758, Val Loss: 1.2278051018714904, Duration: 70.87 sec


In [6]:
training_time = 0
losses = []
val_losses = []

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(epochs):
    model2.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for i, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer2.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model2(inputs)
            loss = criterion(outputs, labels)

        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer2)
        clip_grad_norm_(model2.parameters(), max_norm=1.0)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer2)
        scaler.update()

        # scheduler.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)
 
    model2.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model2(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, Duration: {epoch_duration:.2f} sec'
    print(text)

Epoch 1: 100%|██████████| 98/98 [00:59<00:00,  1.65it/s]


	Loss: 2.1756037838604985, Val Loss: 1.834579074382782, Duration: 70.78 sec


Epoch 2: 100%|██████████| 98/98 [00:58<00:00,  1.66it/s]


	Loss: 1.9012131155753622, Val Loss: 1.616855013370514, Duration: 70.50 sec


Epoch 3: 100%|██████████| 98/98 [00:59<00:00,  1.66it/s]


...

	Loss: 1.256564395768302, Val Loss: 0.8532435089349747, Duration: 72.01 sec


Epoch 29: 100%|██████████| 98/98 [00:59<00:00,  1.66it/s]


	Loss: 1.248545028725449, Val Loss: 0.8422702550888062, Duration: 70.89 sec


Epoch 30: 100%|██████████| 98/98 [01:00<00:00,  1.63it/s]


	Loss: 1.2298593484625524, Val Loss: 0.833803978562355, Duration: 72.00 sec


In [7]:
# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model2(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
display(performance_metrics)

,Metric,Value
0,Accuracy,0.707900
1,Precision,0.704416
2,Recall,0.707900
3,F1 Score,0.701808


In [16]:
training_time = 0
losses = []
val_losses = []

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(epochs):
    model2.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for i, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer2.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model2(inputs)
            loss = criterion(outputs, labels)

        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer2)
        clip_grad_norm_(model2.parameters(), max_norm=1.0)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer2)
        scaler.update()

        # scheduler.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)
 
    model2.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model2(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, Duration: {epoch_duration:.2f} sec'
    print(text)

Epoch 1: 100%|██████████| 98/98 [00:59<00:00,  1.65it/s]


	Loss: 2.154040258757922, Val Loss: 1.8567254722118378, Duration: 71.37 sec


Epoch 2: 100%|██████████| 98/98 [00:59<00:00,  1.65it/s]


	Loss: 1.9650883358352038, Val Loss: 1.6870384693145752, Duration: 71.07 sec


Epoch 3: 100%|██████████| 98/98 [00:59<00:00,  1.64it/s]


...

	Loss: 1.489033976379706, Val Loss: 1.14200758934021, Duration: 71.31 sec


Epoch 29: 100%|██████████| 98/98 [00:59<00:00,  1.65it/s]


	Loss: 1.489051557317072, Val Loss: 1.1341753959655763, Duration: 71.07 sec


Epoch 30: 100%|██████████| 98/98 [00:59<00:00,  1.64it/s]


	Loss: 1.4900078870812241, Val Loss: 1.1141150832176208, Duration: 71.47 sec


In [19]:
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model2(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
display(performance_metrics)

,Metric,Value
0,Accuracy,0.617900
1,Precision,0.612416
2,Recall,0.617900
3,F1 Score,0.609722


In [7]:
training_time = 0
losses = []
val_losses = []

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(epochs):
    model3.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for i, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer3.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model3(inputs)
            loss = criterion(outputs, labels)

        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer3)
        clip_grad_norm_(model3.parameters(), max_norm=1.0)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer3)
        scaler.update()
        # scheduler.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

 
    model3.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model3(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, Duration: {epoch_duration:.2f} sec'
    print(text)

Epoch 1: 100%|██████████| 98/98 [00:57<00:00,  1.69it/s]


	Loss: 2.1512244745176665, Val Loss: 1.9292829930782318, Duration: 69.18 sec


Epoch 2: 100%|██████████| 98/98 [00:57<00:00,  1.70it/s]


	Loss: 2.0759885895008945, Val Loss: 1.8493571758270264, Duration: 69.22 sec


Epoch 3: 100%|██████████| 98/98 [00:57<00:00,  1.69it/s]


...

	Loss: 1.5737232152296572, Val Loss: 1.230663114786148, Duration: 69.08 sec


Epoch 29: 100%|██████████| 98/98 [00:57<00:00,  1.70it/s]


	Loss: 1.5672574213572912, Val Loss: 1.2430946111679078, Duration: 69.35 sec


Epoch 30: 100%|██████████| 98/98 [00:58<00:00,  1.69it/s]


	Loss: 1.5567029155030543, Val Loss: 1.2272435903549195, Duration: 69.74 sec


In [16]:
training_time = 0
losses = []
val_losses = []

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(epochs):
    model4.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for i, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer4.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model4(inputs)
            loss = criterion(outputs, labels)

        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer4)
        clip_grad_norm_(model4.parameters(), max_norm=1.0)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer4)
        scaler.update()

        # scheduler.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

 
    model4.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model4(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, Duration: {epoch_duration:.2f} sec'
    print(text)

Epoch 1: 100%|██████████| 98/98 [00:57<00:00,  1.70it/s]


	Loss: 2.248373255437734, Val Loss: 2.0745943784713745, Duration: 69.99 sec


Epoch 2: 100%|██████████| 98/98 [00:57<00:00,  1.70it/s]


	Loss: 2.1086200183751633, Val Loss: 1.8571063697338104, Duration: 70.54 sec


Epoch 3: 100%|██████████| 98/98 [00:59<00:00,  1.65it/s]


...

	Loss: 1.4353215682263276, Val Loss: 1.0680390059947968, Duration: 71.09 sec


Epoch 29: 100%|██████████| 98/98 [00:57<00:00,  1.70it/s]


	Loss: 1.43426243139773, Val Loss: 1.0625789761543274, Duration: 70.23 sec


Epoch 30: 100%|██████████| 98/98 [00:58<00:00,  1.68it/s]


	Loss: 1.4235068985394068, Val Loss: 1.0531329423189164, Duration: 70.95 sec


In [17]:
# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model4(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
display(performance_metrics)

,Metric,Value
0,Accuracy,0.626600
1,Precision,0.620472
2,Recall,0.626600
3,F1 Score,0.618190


In [7]:
training_time = 0
losses = []
val_losses = []

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(10):
    model5.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for i, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer5.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model5(inputs)
            loss = criterion(outputs, labels)

        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer5)
        clip_grad_norm_(model5.parameters(), max_norm=1.0)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer5)
        scaler.update()

        # scheduler.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

 
    model5.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model5(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, Duration: {epoch_duration:.2f} sec'
    print(text)

Epoch 1: 100%|██████████| 98/98 [00:58<00:00,  1.69it/s]


	Loss: 1.7436095938390614, Val Loss: 0.742325383424759, Duration: 69.75 sec


Epoch 2: 100%|██████████| 98/98 [00:59<00:00,  1.66it/s]


	Loss: 0.9124265690239108, Val Loss: 0.3213254988193512, Duration: 70.94 sec


Epoch 3: 100%|██████████| 98/98 [00:58<00:00,  1.68it/s]


...

	Loss: 0.49971461022386743, Val Loss: 0.10007617846131325, Duration: 70.85 sec


Epoch 9: 100%|██████████| 98/98 [00:59<00:00,  1.64it/s]


	Loss: 0.48375548391926043, Val Loss: 0.09374292325228453, Duration: 71.89 sec


Epoch 10: 100%|██████████| 98/98 [00:59<00:00,  1.65it/s]


	Loss: 0.47205124430510464, Val Loss: 0.0879520645365119, Duration: 71.46 sec


In [10]:
# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model5(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
display(performance_metrics)

,Metric,Value
0,Accuracy,0.971000
1,Precision,0.970981
2,Recall,0.971000
3,F1 Score,0.970976
